In [3]:
import torch
import torch.nn as nn

C:\Users\USER\anaconda3\envs\env\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\USER\anaconda3\envs\env\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\USER\anaconda3\envs\env\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\Users\USER\anaconda3\envs\env\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
class CharCNN(nn.Module):
    def __init__(self,                          # CharCNN 클래스의 인스턴스를 생성했을 때 갖게 되는 성질을 정의하는 메서드
                max_word_len = 10,
                kernel_lst = "2,3,4",
                num_filters=32,
                char_vocab_size = 1000,
                char_emb_dim = 30,
                final_char_dim = 50
                ):
        super(CharCNN, self).__init__()         #nn.Module 내에 있는 메서드를 상속받아 이용한다.

        # Initialize character embedding
        self.char_emb = nn.Embedding(char_vocab_size, char_emb_dim, padding_idx=0)
        nn.init.uniform_(self.char_emb.weight, -0.25, 0.25)

        kernel_lst = list(map(int, kernel_lst.split(",")))          # "2, 3, 4" -> [2, 3, 4]

        # Convolution for each kernel
        self.convs = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(char_emb_dim, num_filters, kernel_size, padding=kernel_size // 2),
                nn.Tanh(),      # As the paper mentioned
                nn.MaxPool1d(max_word_len - kernel_size + 1),
                nn.Dropout(0.25),       # As same as the original code implementation
            ) for kernel_size in kernel_lst
        ])

        self.linear = nn.Sequential(
            nn.Linear(num_filters * len(kernel_lst), 100),
            nn.ReLU(),          # As same as the original code implementation
            nn.Dropout(0.25),
            nn.Linear(100, final_char_dim)
        )

    def forward(self, x):
        """
        :param x: (batch_size, max_seq_len, max_word_len)
        :return: (batch_size, max_seq_len, final_char_dim)
        """
        batch_size = x.size(0)
        max_seq_len = x.size(1)
        max_word_len = x.size(2)

        x = self.char_emb(x)        # (b, s, w, d)
        x = x.view(batch_size * max_seq_len, max_word_len, -1)      # (b*s, w, d)
        x = x.transpose(2, 1)       # (b*s, d, w): Conv1d takes in (batch, dim, seq_len), but raw embedded is (batch, seq_len, dim)

        conv_lst = [conv(x) for conv in self.convs]
        conv_concat = torch.cat(conv_lst, dim = -1)     # (b*s, num_filters, len(kernel_lst))
        conv_concat = conv_concat.view(conv_concat.size(0), -1)     # (b*s, num_filters * len(kernel_lst))

        output = self.linear(conv_concat)       # (b*s, final_char_dim)
        output = output.view(batch_size, max_seq_len, -1)   # (b, s, final_char_dim)
        return output